# Masterthesis

In [ ]:
import networkx as nx
import markovmixing as mkm
import numpy as np
import random

execfile('graph_util.py')

## Generate and save some random graphs

In [ ]:
random.seed(1)

G = erdos_renyi_giant_component(2500000,1.1/2500000)
print nx.number_of_nodes(G)

G_2core = nx.k_core(G,k=2)
print nx.number_of_nodes(G_2core)

nx.write_sparse6(G_2core, 'graphs/ergc_2core.s6')

In [ ]:
# verify that the relation of giant/2-core/kernel is indeed typical for these parameters
for i in [2,3,4,5]:
    random.seed(i)
    G = erdos_renyi_giant_component(2500000,1.1/2500000)
    print nx.number_of_nodes(G)
    print nx.degree_histogram(G)

    G_2core = nx.k_core(G,k=2)
    print nx.number_of_nodes(G_2core)
    
    G_kernel = pseudo_kernel(G)
    print nx.number_of_nodes(G_kernel)

In [ ]:
random.seed(10)

G = erdos_renyi_giant_component(250000,1.1/250000)

nx.write_sparse6(G, 'graphs/ergc.s6')

In [ ]:
# verify that the relation of giant/2-core/kernel is indeed typical for these parameters
for i in [0,1,2,3]:
    random.seed(i)
    G = erdos_renyi_giant_component(250000,1.1/250000)
    print nx.number_of_nodes(G)

    G_2core = nx.k_core(G,k=2)
    print nx.number_of_nodes(G_2core)
    
    G_kernel = pseudo_kernel(G)
    print nx.number_of_nodes(G_kernel)

## Determine mixing times on different types of graphs

### Lazy random walk on the 1000-path

In [ ]:
G = nx.path_graph(1000)

mc = mkm.nx_graph_lazy_srw(G)

for i in [0,250,500]:
    d = np.zeros(1000)
    d[i] = 1.
    mc.add_distributions(d)
    
mc.plot_tv_mixing(y_tol=0.01, threshold=0.05, text=False)

### Lazy biased random walk on the 1000-path

In [ ]:
mc = mkm.MarkovChain(mkm.line_lazy_transition_matrix(1000, p=0.51))

for i in [0,500,999]:
    d = np.zeros(1000)
    d[i] = 1.
    mc.add_distributions(d)
    
mc.plot_tv_mixing(y_tol=0.01, threshold=1e-5, text=False)

### Lazy random walk on the 50-cycle

In [ ]:
G = nx.cycle_graph(50)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on the 50-cycle with appended binary trees of height 8

In [ ]:
G = append_graph_to_all_nodes(nx.cycle_graph(50), nx.balanced_tree(2,8), 0)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on the 15-dimensional hypercube

In [ ]:
mc = mkm.MarkovChain(mkm.hypercube_transition_matrix(15))
mc.add_random_delta_distributions(1)
mc.set_stationary(mkm.uniform_distribution(mc.get_n()))

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### SRW on a random 6-regular graph with n=50.000

In [ ]:
G = nx.read_sparse6('graphs/6_regular.s6')

mc = mkm.nx_graph_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on an Erdős–Rényi giant component, it's 2-core and pseudo-kernel

In [ ]:
# giant component
G = nx.read_sparse6('graphs/ergc.s6')

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

In [ ]:
# 2-core
G = nx.k_core(nx.read_sparse6('graphs/ergc.s6'),k=2)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

In [ ]:
# pseudo-kernel
G = pseudo_kernel(nx.read_sparse6('graphs/ergc.s6'))

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Lazy random walk on the 2-core and kernel of an Erdős–Rényi giant component

In [ ]:
G = nx.read_sparse6('graphs/ergc_2core.s6')

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

In [ ]:
G = pseudo_kernel(nx.read_sparse6('graphs/ergc_2core.s6'))

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

### Another graph that should have cutoff

In [ ]:
random.seed(25)
k = 10

G = nx.empty_graph(k)

for i in xrange(50000-k):
    G.add_node(G.number_of_nodes())
    
    for j in xrange(k):
        v = random.randint(0,G.number_of_nodes()-2)
        G.add_edge(v,G.number_of_nodes()-1)

#show_graph(G)

mc = mkm.nx_graph_lazy_srw(G)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

In [ ]:
random.seed(2)

n = 100000
lam = 1.1

G = erdos_renyi_giant_component(n,lam/n)
K = pseudo_kernel(G)

print nx.number_of_nodes(G)
print nx.number_of_nodes(K)
print ((100.*nx.number_of_nodes(K))/nx.number_of_nodes(G))

mc = mkm.nx_graph_lazy_srw(K)
mc.add_random_delta_distributions(1)

mc.plot_tv_mixing(y_tol=0.01, threshold=0.01, text=False)

## Draw some graphs

### 10-path with attached Galton-Watson trees

In [ ]:
np.random.seed(0)

G = nx.path_graph(10)
grow_gw_trees_at_all_nodes(G, lambda: np.random.poisson(0.9, 1))

pos = nx.graphviz_layout(G, prog='neato')
nx.draw(G, pos, with_labels=False, arrows=False, node_size=100, node_color='k', edge_color='k', width=1.5)
plt.show()

pos = nx.graphviz_layout(G, prog='neato')
nx.draw(G, pos, with_labels=True, arrows=False)
plt.show()


### 2-core and path contraction for a giant component

In [ ]:
random.seed(0)

n = 100000
lam = 1.1

G = erdos_renyi_giant_component(n,lam/n)
K = pseudo_kernel(G)

print nx.number_of_nodes(G)
print nx.number_of_nodes(K)

pos = nx.graphviz_layout(K, prog='fdp')
nx.draw(K, pos, with_labels=False, arrows=False, node_size=100, node_color='k', edge_color='k', width=1.5)
plt.show()

### Two Galton-Watson trees with different distribution to illustrate couplings

In [ ]:
# make the result look like a coupling of trees was tried, too
np.random.seed(11)

G = grow_gw_tree(lambda: np.random.choice([1,1,1,1,1,2,2,2,3,3]), n_gen=5)
show_tree(G)

# two nodes of degree 2 become degree 3 to reflect the different distribution
G.add_edge(6, G.number_of_nodes())
G.add_edge(35, G.number_of_nodes())
G.add_edge(G.number_of_nodes()-1, G.number_of_nodes())
G.add_edge(G.number_of_nodes()-2, G.number_of_nodes())
G.add_edge(G.number_of_nodes()-1, G.number_of_nodes())
G.add_edge(G.number_of_nodes()-3, G.number_of_nodes())
G.add_edge(G.number_of_nodes()-4, G.number_of_nodes())
show_tree(G)

### A giant component and it's kernel

In [ ]:
random.seed(2)

n = 2000
lam = 1.5

G = erdos_renyi_giant_component(n,lam/n)

print nx.number_of_nodes(G)

pos = nx.graphviz_layout(G, prog='fdp')
nx.draw(G, pos, with_labels=False, arrows=False, node_size=40, node_color='k', edge_color='k', width=0.8)
plt.show()

In [ ]:
K = pseudo_kernel(G)

print nx.number_of_nodes(K)

pos = nx.graphviz_layout(K, prog='fdp')
nx.draw(K, pos, with_labels=False, arrows=False, node_size=40, node_color='k', edge_color='k', width=0.8)
plt.show()

## Plot of the degree distribution of a random graph and it's giant component

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.stats import binom

n = 1000000
p = 1.05/n

random.seed(3)

# sampe random graph and extract it's giant component
G = nx.fast_gnp_random_graph(n,p)

number_of_nodes = 0
C_1 = 0
for c in nx.connected_component_subgraphs(G):
    if nx.number_of_nodes(c) > number_of_nodes:
        number_of_nodes = nx.number_of_nodes(c)
        C_1 = c
        
# degree distributions
d1 = np.array(nx.degree_histogram(G))
d2 = np.array(nx.degree_histogram(C_1))

print sum(d1)
print d1
print sum(d2)
print d2
print nx.number_of_nodes(C_1)
print np.exp(pow(np.log(n),1./3))

In [ ]:
print np.exp(pow(np.log(n),1./3))

In [ ]:
# random graph
x = np.arange(8)
y = (d1/(1.0*sum(d1)))[0:8]
    
fig = plt.figure()

vmax = np.max(y)
vmin = (np.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(x, y, color=colormap.to_rgba(y), align='edge', width=0.8)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()

In [ ]:
# giant component
x = np.arange(8)
y = (d2/(1.0*sum(d2)))[0:8]
    
fig = plt.figure()

vmax = np.max(y)
vmin = (np.min(y)*3. - vmax)/2.

colormap = ScalarMappable(norm=Normalize(vmin, vmax), cmap='Blues')

plt.bar(x, y, color=colormap.to_rgba(y), align='edge', width=0.8)

plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')

plt.xlim(0, len(x))
plt.show()

## Speed of random walk on a Poisson-Galton-Watson tree

In [ ]:
from scipy.optimize import brentq
from scipy.misc import factorial

import matplotlib.pyplot as plt

def poi_gw_q(lam):
    res = brentq(lambda x: pow(np.e, lam*(x-1))-x, 0, 1-1e-10, xtol=1e-12)
    return res

def poi_gw_nu(lam):
    q = poi_gw_q(lam)
    x = []
    k = 1
    
    while True:
        a = sum(x)
    
        for i in xrange(10):
            x.append(pow(lam,k)/factorial(k)*(1-pow(q,k+1))/(1-pow(q,2))*(k-1)/(k+1))
            k = k+1
            
        if (sum(x)-a)/sum(x) < 1e-12:
            break
        
    return pow(np.e,-lam)*sum(x)

In [ ]:
# plot the extinction probability as a function of lambda
x = np.arange(1,7,0.01)
y = []

for xx in x:
    y.append(poi_gw_q(xx))

plt.plot(x, y)
plt.show()

In [ ]:
# plot speed of random walk as a function of lambda
x = np.arange(1,7,0.01)
y = []

for xx in x:
    y.append(poi_gw_nu(xx))

plt.xlim(1, 7)
plt.plot(x, y)
plt.show()